<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/Gena/palettes_crameri_oleron_dem.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/Gena/palettes_crameri_oleron_dem.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/Gena/palettes_crameri_oleron_dem.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://geemap.org). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [ ]:
import ee
import geemap

## Create an interactive map 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map

## Add Earth Engine Python script 

In [ ]:
# Add Earth Engine dataset
from ee_plugin.contrib import palettes

dem = ee.Image("AHN/AHN2_05M_RUW").convolve(ee.Kernel.gaussian(0.5, 0.3, 'meters'))

extrusion = 3
weight = 0.7
palette = palettes.crameri['oleron'][50]

rgb = dem.visualize(**{'min': 0, 'max': 3, 'palette': palette })
hsv = rgb.unitScale(0, 255).rgbToHsv()
hs = ee.Terrain.hillshade(dem.multiply(extrusion), 315, 35).unitScale(0, 255)
hs = hs.multiply(weight).add(hsv.select('value').multiply(1 - weight))
saturation = hsv.select('saturation').multiply(0.5)
hsv = hsv.addBands(hs.rename('value'), ['value'], True)
hsv = hsv.addBands(saturation, ['saturation'], True)
rgb = hsv.hsvToRgb()

# rgb = rgb.updateMask(dem.unitScale(0, 3))

Map.addLayer(rgb, {}, 'Dutch AHN DEM', True)
Map.setCenter(4.5618, 52.1664, 18)


## Display Earth Engine data layers 

In [ ]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map